In [1]:
!pip3 install plotly


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [3]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

torch.manual_seed(42) # Set for reproducability

In [4]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
import sys
from tqdm import tqdm
import time
import copy

In [6]:
# Constants
DATA_DIR = "../data/ship_100train_500val_200test"
DATA_SPLIT = [0.7, 0.2, 0.1]
BATCH_SIZE = 256
LR = 0.001
NUM_WORKERS = os.cpu_count()


In [7]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir + '/train')
    return all_data.classes

In [8]:
classes = get_classes(DATA_DIR)
print(classes, len(classes))

['ContainerShip', 'Cruise', 'Tanker', 'Warship'] 4


In [9]:
def get_data_loader(data_dir, batch_size, ds_type, ds_split):
    accepted_types = ('train', 'val', 'test')
    assert ds_type in accepted_types, "Invalid Dataset Type"
    
    full_dataset = datasets.ImageFolder(data_dir)
    full_dataset_size = len(full_dataset)
    
    assert len(ds_split) == 3, "ds_split does not have 3 sections"
    assert round(sum(ds_split)) == 1, "ds_split ratio does not add to 1"
    
    train_size = int(full_dataset_size * ds_split[0])
    valid_test_size = full_dataset_size - train_size
    
    train_dataset, val_test_dataset = data.random_split(full_dataset, [train_size, valid_test_size])
    
    if ds_type == "train":
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random'),
        ])
        train_dataset.dataset.transform = transform
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
        return train_loader, len(train_dataset)
    
    else:
        val_size = int(full_dataset_size * ds_split[1])
        test_size = valid_test_size - val_size
        
        transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        
        val_test_dataset.dataset.transform = transform
        
        # Further split
        val_dataset, test_dataset = data.random_split(val_test_dataset, [val_size, test_size])
        
        if ds_type == "val":
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
            return val_loader, len(val_dataset)
            
        elif ds_type == "test":
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
            return test_loader, len(test_dataset)

In [10]:
def get_data_loader(data_dir, batch_size, ds_type):
    
    
    if ds_type == "train":
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random'),
        ])
        img_dataset = datasets.ImageFolder(os.path.join(data_dir, ds_type), transform=transform)
    
    else:
        transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        img_dataset = datasets.ImageFolder(os.path.join(data_dir, ds_type), transform=transform)
        
    data_loader = DataLoader(img_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
    
    return data_loader, len(img_dataset)
    
        
    

In [11]:
train_loader, train_size = get_data_loader(DATA_DIR, BATCH_SIZE, "train")
val_loader, val_size = get_data_loader(DATA_DIR, BATCH_SIZE, "val")
test_loader, test_size = get_data_loader(DATA_DIR, BATCH_SIZE, "test")

In [12]:
dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test" : test_loader,
}

dataset_sizes = {
    "train": train_size,
    "val": val_size,
    "test": test_size,
}

In [13]:
print(f"Training Set has {train_size} data")
print(f"Val Set has {val_size} data")
print(f"Test Set has {test_size} data")
print(f"Train {len(train_loader)} batches, Val {len(val_loader)} batches, Test {len(test_loader)} batches")

Training Set has 400 data
Val Set has 2000 data
Test Set has 800 data
Train 2 batches, Val 8 batches, Test 4 batches


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
model = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [16]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
for param in model.features.parameters(): # freeze model
    param.requires_grad = False
    
n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(classes))
model.classifier[6] = last_layer
    
# for param in model.parameters(): # freeze model
#     param.requires_grad = False
    
# n_inputs = model.fc.in_features
# model.fc = nn.Linear(n_inputs, len(classes))


model = model.to(device)

print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)


In [19]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [20]:
logs = df = pd.DataFrame(columns=['epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc'])
logs['epoch'] = df['epoch'].astype('int')

In [21]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-"*10)
        
        # Set model to correct mode for training/validation
        for phase in ['train', 'val']:
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0.0
        
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase == "train"): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == "train":
                scheduler.step() # step at end of epoch

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == "train":
                train_loss = epoch_loss
                train_acc = epoch_acc.item()
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc.item()

            print(f"{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
                
        logs.loc[len(logs.index)] = [epoch, train_loss, train_acc, val_loss, val_acc] 
        print()
        
    time_elapsed = time.time() - since
    print(f"Training completed in {time_elapsed // 60:.0f}m, {time_elapsed % 60:.0f}s")
    print(f"Best Val Acc: {best_acc:.4f}")
    
    model.load_state_dict(best_model_wts)
    return model, logs
    
            

In [22]:
model_ft, logs = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.89s/it]


train Loss: 1.3353, Acc: 0.3725


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.03it/s]


val Loss: 0.7716, Acc: 0.7360

Epoch 1/199
----------


...

val Loss: 0.3586, Acc: 0.9070

Epoch 150/199
----------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.44s/it]


train Loss: 0.0329, Acc: 0.9850


  0%|                                                                                                                                                   | 0/8 [2:22:34<?, ?it/s]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
logs

In [ ]:
# logs.to_csv('alexnet_logs_200train.csv', index=False)

In [ ]:
fig = go.Figure()

# Add Traces
fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['train_loss'],
               name='Train Loss',
               line=dict(color='firebrick', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['train_acc'],
               name='Train Acc',
               line=dict(color='green', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['val_loss'],
               name='Val Loss',
               line=dict(color='blue', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['val_acc'],
               name='Val Acc',
               line=dict(color='blue', width=2)))

# Edit the layout
fig.update_layout(title='Pretext Training',
                   xaxis_title='Epochs',
                   yaxis_title='Loss')

#Add Buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.7,
            y=1.22,
            buttons=list([
                dict(label="Alexnet",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Pretext Analysis",}
                        ]
                    ),
                dict(label="Loss",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Train:Val Loss",}
                        ]
                    ),
                dict(label="Acc",
                     method="update",
                     args=[
                           {"visible": [False, True, False, True]},
                           {"title": "Train:Val Acc",}
                        ]
                    ),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="Alexnet Supervised",
    xaxis_domain=[0.05, 1.0],
    xaxis_title="Epochs",
    yaxis_title="Loss/Acc",
)

fig.show()


In [ ]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model.to(device)
model.eval()

for data, target in tqdm(test_loader, leave=True):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == BATCH_SIZE:
        for i in range(BATCH_SIZE):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            
test_loss = test_loss / test_size
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print(f"Test Accuracy of {classes[i]}: {100*class_correct[i]/class_total[i]:.2f}% ({class_correct[i]}/{class_total[i]})")
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))

print(f"Test Accuracy of {100*np.sum(class_correct)/np.sum(class_total):.2f}% ({np.sum(class_correct)}/{np.sum(class_total)})")

In [ ]:
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("ship.pt")